### Description
This tutorial shows a complete model process of one cell type from:
* Data preparation
* Data preprocess
* Model training, test performance evaluation
* Independent test prediction

### 0. Input
* Reference genome
* Cell type

In [1]:
# Select your reference genome
ref_genome = 'hg19' # or 'hg38'

# Select your cell type (here Roadmap id)
cell_type = 'E034'


### 1. Prepare data
* Get reference genome
* This example shows the path of hg19 build from UCSC

In [2]:
# get reference genome
! mkdir -p data/reference_genome/
! wget https://hgdownload.soe.ucsc.edu/goldenPath/{ref_genome}/bigZips/{ref_genome}.fa.gz
! mv {ref_genome}.fa.gz data/reference_genome/
! gunzip data/reference_genome/{ref_genome}.fa.gz

--2020-10-15 11:13:09--  https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948731419 (905M) [application/x-gzip]
Saving to: ‘hg19.fa.gz’

hg19.fa.gz            0%[                    ]   2.34M   889KB/s               ^C

gzip: data/reference_genome/hg19.fa.gz: unexpected end of file


-----------------
* Generate 200-bp bed and fasta file from reference genome
* Output: `data/200bp_bin.bed`. `data/200bp_bin.fa` 

In [2]:
! python preprocess/get_sequences.py --genome data/genome/{ref_genome}.fa --out data/


Feature (chr1:249250600-249250800) beyond the length of chr1 size (249250621 bp).  Skipping.
Feature (chr2:243199200-243199400) beyond the length of chr2 size (243199373 bp).  Skipping.
Feature (chr3:198022400-198022600) beyond the length of chr3 size (198022430 bp).  Skipping.
Feature (chr4:191154200-191154400) beyond the length of chr4 size (191154276 bp).  Skipping.
Feature (chr5:180915200-180915400) beyond the length of chr5 size (180915260 bp).  Skipping.
Feature (chr6:171115000-171115200) beyond the length of chr6 size (171115067 bp).  Skipping.
Feature (chr7:159138600-159138800) beyond the length of chr7 size (159138663 bp).  Skipping.
Feature (chr8:146364000-146364200) beyond the length of chr8 size (146364022 bp).  Skipping.
Feature (chr9:141213400-141213600) beyond the length of chr9 size (141213431 bp).  Skipping.
Feature (chr10:135534600-135534800) beyond the length of chr10 size (135534747 bp).  Skipping.
Feature (chr11:135006400-135006600) beyond the length of chr11 size

* Get H3K27ac and DNase narrowPeak files of a specific cell type
* This example shows the Roadmap path of BLD.CD3.PPC data (E034)

In [3]:
# get peaks files
! mkdir -p data/peakfiles
! wget https://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/{cell_type}-H3K27ac.narrowPeak.gz
! wget https://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/{cell_type}-DNase.macs2.narrowPeak.gz
! mv {cell_type}* data/peakfiles/

--2020-10-15 11:13:37--  https://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/E034-H3K27ac.narrowPeak.gz
Resolving egg2.wustl.edu (egg2.wustl.edu)... 128.252.187.85
Connecting to egg2.wustl.edu (egg2.wustl.edu)|128.252.187.85|:443... connected.
^C
--2020-10-15 11:13:38--  https://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/E034-DNase.macs2.narrowPeak.gz
Resolving egg2.wustl.edu (egg2.wustl.edu)... 128.252.187.85
Connecting to egg2.wustl.edu (egg2.wustl.edu)|128.252.187.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5671786 (5.4M) [application/x-gzip]
Saving to: ‘E034-DNase.macs2.narrowPeak.gz’

034-DNase.macs2.nar   8%[>                   ] 481.99K   216KB/s               ^C


### 2. Preprocess
* Generate positive and negative samples (1:10 in train, 1:5 in test)

In [6]:
! python preprocess/process_narrowpeaks.py \
        --h3k27ac_file data/peakfiles/{cell_type}-H3K27ac.narrowPeak.gz \
        --epi_file data/peakfiles/{cell_type}-DNase.macs2.narrowPeak.gz \
        --hg38_fasta_bins data/200bp_bin.fa \
        --hg38_bed_bins data/200bp_bin.bed \
        --output_name {cell_type}_out

### Now fetching signal values from h3k27ac bed files...
### Now fetching signal values from dnase bed files...
### Now selecting negative seq from hg38 200bp bins pool...
Processed 0 negative sequences...
Processed 324856 negative sequences...
Processed 649712 negative sequences...
Processed 974568 negative sequences...
Processed 1299424 negative sequences...
Processed 1624280 negative sequences...
Processed 1949136 negative sequences...
Processed 2273992 negative sequences...
Processed 2598848 negative sequences...
Processed 2923704 negative sequences...
### Now processing positive bins...
Processed 50000 positive sequences for selected dnase...
Processed 100000 positive sequences for selected dnase...
Processed 150000 positive sequences for selected dnase...
Processed 200000 positive sequences for selected dnase...
Processed 250000 positive sequences for selected dnase...
Processed 300000 positive sequences for selected dnase...
### Now processing negative bins...
Processed 50000 po

---------------
* Pack to .h5 file as model input

In [7]:
!python preprocess/generate_h5.py --data_prefix data/single_cell_type/{cell_type}_out


# === Creating a Training, Test and Validation Set from provided input === #

Reading lines ...

Converting to binary representation:

###-- Epigenetic Feature Length = 25 --###

Converting to binary representation:

Sampled into sets ...

Storing Coordinates ...

Initializing hdf5 Storage Files ...

Running through raw file again, converting sequences and store in sets ...
Converting and storing sequences of length 200 bp.
Written lines ... 0
Written lines ... 10000
Written lines ... 20000
Written lines ... 30000
Written lines ... 40000
Written lines ... 50000
Written lines ... 60000
Written lines ... 70000
Written lines ... 80000
Written lines ... 90000
Written lines ... 100000
Written lines ... 110000
Written lines ... 120000
Written lines ... 130000
Written lines ... 140000
Written lines ... 150000
Written lines ... 160000
Written lines ... 170000
Written lines ... 180000
Written lines ... 190000
Written lines ... 200000
Written lines ... 210000
Written lines ... 220000
Written li

Written lines ... 3030000
Written lines ... 3040000
Written lines ... 3050000
Written lines ... 3060000
Written lines ... 3070000
Written lines ... 3080000
Written lines ... 3090000
Written lines ... 3100000
Written lines ... 3110000
Written lines ... 3120000
Written lines ... 3130000
Written lines ... 3140000
Written lines ... 3150000
Written lines ... 3160000
Written lines ... 3170000
Written lines ... 3180000
Written lines ... 3190000
Written lines ... 3200000
Written lines ... 3210000
Written lines ... 3220000
Written lines ... 3230000
Written lines ... 3240000
Written lines ... 3250000
Written lines ... 3260000
Written lines ... 3270000
Written lines ... 3280000
Written lines ... 3290000
Written lines ... 3300000
Written lines ... 3310000
Written lines ... 3320000
Written lines ... 3330000
Written lines ... 3340000
Written lines ... 3350000
Written lines ... 3360000
Written lines ... 3370000
Skipped 0 elements with sequence length != 200
Converting and storing sequences of length 

### 3. Model
* Train from the .h5 train datasete for 15 epochs with deephaem pretrained model
* Test from the .h5 test dataset
* If gpu devices available, `CUDA_VISIBLE_DEVICES` with known number can be assigned to accelerate the training speed.

In [4]:
! CUDA_VISIBLE_DEVICES=0,1 python accuEnhancer.py \
    --in_file data/single_cell_type/{cell_type}_out_dnase_dataset.h5 \
    --out_name data/{cell_type} \
    --epoch 15 \
    --deephaem_model_path models/deephaem_erythroid_saved_conv_weights.npz

Using TensorFlow backend.

Start to read from h5 files to Numpy array:
Finished reading training....


training_data
(3378503, 200, 4)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2020-10-16 02:52:55.793485: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-16 02:52:55.835254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-16 02:52:55.835620: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2020-10-16 02:52:55.835657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

Train from the beginning!
---- Training 0 iteration------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3209577 samples, validate on 168926 samples
Epoch 1/2
2020-10-16 02:53:05.717351: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-16 02:53:06.285130: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
3209577/3209577 [==============================] - 216s 67us/step - loss: 0.1732 - accuracy_m: 0.9580 - recall_m: 0.8453 - precision_m: 0.7428 - f1_m: 0.7895 - recall_keras: 0.8453 - precision_keras: 0.7428 - f1_keras: 0.7895 - val_loss: 0.0477 - val_accuracy_m: 0.9816 - val_recall_m: nan - val_precision_m: 0.0000e+00 - val_f1_m: nan - val_recall_keras: 0.0000e+00 - val_precision_keras: 0.0000e+00 - val_f1_keras: 0.0000e+00
Epoch 2/2
3209577/3209577 [==============================] - 2

#### If the training was interrupted, load the checkpoint from last epcoh: from checkpoint 0

In [9]:
! CUDA_VISIBLE_DEVICES=0,1 python accuEnhancer.py \
    --in_file data/single_cell_type/{cell_type}_out_dnase_dataset.h5 \
    --out_name data/{cell_type} \
    --epoch 15 \
    --train_from_ckpt 0 

Using TensorFlow backend.

Start to read from h5 files to Numpy array:
Finished reading training....


training_data
(3378503, 200, 4)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2020-10-16 04:05:00.849849: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-16 04:05:00.892260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-16 04:05:00.892696: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2020-10-16 04:05:00.892739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

Checkpoint 0 weights are loaded.
---- Training 1 iteration------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3209577 samples, validate on 168926 samples
Epoch 1/2
2020-10-16 04:05:11.323141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-16 04:05:11.913860: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
3209577/3209577 [==============================] - 216s 67us/step - loss: 0.1434 - accuracy_m: 0.9610 - recall_m: 0.8468 - precision_m: 0.7557 - f1_m: 0.7982 - recall_keras: 0.8468 - precision_keras: 0.7557 - f1_keras: 0.7982 - val_loss: 0.0710 - val_accuracy_m: 0.9710 - val_recall_m: nan - val_precision_m: 0.0000e+00 - val_f1_m: nan - val_recall_keras: 0.0000e+00 - val_precision_keras: 0.0000e+00 - val_f1_keras: 0.0000e+00
Epoch 2/2
3209577/3209577 [============================

#### from checkpoint 9

In [2]:
! CUDA_VISIBLE_DEVICES=0,1 python accuEnhancer.py \
    --in_file data/single_cell_type/{cell_type}_out_dnase_dataset.h5 \
    --out_name data/{cell_type} \
    --epoch 15 \
    --train_from_ckpt 9

Using TensorFlow backend.

Start to read from h5 files to Numpy array:
Finished reading training....


training_data
(3378503, 200, 4)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2020-10-16 15:55:10.715264: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-16 15:55:10.749709: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-16 15:55:10.750108: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2020-10-16 15:55:10.750146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

Checkpoint 9 weights are loaded.
---- Training 10 iteration------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3209577 samples, validate on 168926 samples
Epoch 1/2
2020-10-16 15:55:21.266634: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-16 15:55:21.830169: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
3209577/3209577 [==============================] - 216s 67us/step - loss: 0.1475 - accuracy_m: 0.9609 - recall_m: 0.8423 - precision_m: 0.7566 - f1_m: 0.7967 - recall_keras: 0.8423 - precision_keras: 0.7566 - f1_keras: 0.7967 - val_loss: 0.0517 - val_accuracy_m: 0.9788 - val_recall_m: nan - val_precision_m: 0.0000e+00 - val_f1_m: nan - val_recall_keras: 0.0000e+00 - val_precision_keras: 0.0000e+00 - val_f1_keras: 0.0000e+00
Epoch 2/2
3209577/3209577 [===========================

### Independent prediction
* Self-prepared 200-bp bed file (and/or fasta) as input

In [5]:
! cat data/test.bed

chr1	114377468	114377668
chr1	713000	713200


In [6]:
! bedtools getfasta -fi data/genome/hg19.fa -bed data/test.bed -fo data/test.fa.out
! cat data/test.fa.out

>chr1:114377468-114377668
ttaaaggcatgagccaccatgcccATCCCACACTTTATTTTATACTTACTGAACTGTACTCACCAGCTTCCTCAACCACAATAAATGATTCAGGTGTCCATACAGGAAGTGGAGGGGGGATTTCATCATCTATCCTTGGAGCAGTTGCTATCCAAAATGTCAAAAATATTGTAACAATTGTTAATTAGAACAATCCAAAG
>chr1:713000-713200
ggctggggtgcagtggtgtgatcttggcccagtgcaacctctgcctcccgggttcaagtaattctcctttatcagcctcccaggtagctgggactacaggcatgcgccaccacggccagctaatttttgtattttttgtagagactgggtttcaccatggccaggctggtctccaactcctgacctcaggtgatccaccc


----------------------
* Get corresponding dnase feature and H3K27ac

In [7]:
cell_type = 'E034'
h3k27ac_file = f'data/peakfiles/{cell_type}-H3K27ac.narrowPeak.gz'
epi_file = f'data/peakfiles/{cell_type}-DNase.macs2.narrowPeak.gz'

* Generate test data in the format of `<chr> <start> <end> <comma separated IDs> <raw sequence> <epi mark>`

In [8]:
!python preprocess/process_narrowpeaks_test.py \
        --h3k27ac_file {h3k27ac_file} \
        --epi_file {epi_file} \
        --hg38_fasta_bins data/test.fa.out \
        --hg38_bed_bins data/test.bed \
        --output_name {cell_type}_test_out

### Now fetching signal values from h3k27ac bed files...
### Now fetching signal values from dnase bed files...
### Now processing positive bins...


* check the generated test file

In [12]:
! cat data/single_cell_type/{cell_type}_test_out_dnase.test.dat

chr1	114377468	114377668	0	ttaaaggcatgagccaccatgcccATCCCACACTTTATTTTATACTTACTGAACTGTACTCACCAGCTTCCTCAACCACAATAAATGATTCAGGTGTCCATACAGGAAGTGGAGGGGGGATTTCATCATCTATCCTTGGAGCAGTTGCTATCCAAAATGTCAAAAATATTGTAACAATTGTTAATTAGAACAATCCAAAG	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	12.91072	12.91072	0.0	0.0	0.0	0.0	0.0	0.0	0.0
chr1	713000	713200	1	ggctggggtgcagtggtgtgatcttggcccagtgcaacctctgcctcccgggttcaagtaattctcctttatcagcctcccaggtagctgggactacaggcatgcgccaccacggccagctaatttttgtattttttgtagagactgggtttcaccatggccaggctggtctccaactcctgacctcaggtgatccaccc	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	32.05043	32.05043	32.05043	0.0	6.65678	0.0	0.0	0.0	0.0


* Pack to .h5 file for model prediction

In [9]:
!python preprocess/generate_h5_test.py --data_prefix data/single_cell_type/{cell_type}_test_out


# === Creating a Training, Test and Validation Set from provided input === #

Reading lines ...

Converting to binary representation:

Sampled into sets ...

Storing Coordinates ...

Initializing hdf5 Storage Files ...

Running through raw file again, converting sequences and store in sets ...
Converting and storing sequences of length 200 bp.
Written lines ... 0
Skipped 0 elements with sequence length != 200

Saved the data Data.



------------
* Predict test data

In [15]:
!python predict_test.py --in_file data/single_cell_type/{cell_type}_test_out_dnase_dataset.h5 --out_name data/{cell_type}_test


Using TensorFlow backend.

Start to read from h5 files to Numpy array:
test_data
(2, 200, 4)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2020-10-16 17:20:00.442141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-16 17:20:00.483810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-16 17:20:00.484176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.545
pciBusID: 0000:01:00.0
2020-10-16 17:20:00.484211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-16 17:20:00.484550: 


2020-10-16 17:20:02.707880: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-16 17:20:03.214577: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
Predict probability: [[0.00116857]
 [0.99360955]]

test loss: ----

test accuracy:  1.0

test recall:  1.0

test precision:  1.0

test f1_score:  1.0
